In [8]:
import requests
from bs4 import BeautifulSoup
from retry import retry
import urllib
import time
import numpy as np
import pandas as pd
from datetime import date

from sqlalchemy import create_engine
from sqlalchemy import text

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from joblib import dump
from joblib import load

In [9]:
# 複数ページの情報をまとめて取得
data_samples = []

# スクレイピングするページ数
max_page = 100
# SUUMOを桜山のみ指定して検索して出力した画面のurl(ページ数フォーマットが必要)
base_url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=050&bs=040&ra=023&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&ek=323016210&rn=3230'

# リクエストがうまく行かないパターンを回避するためのやり直し
@retry(tries=3, delay=10, backoff=2)
def load_page(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup

# 処理時間を測りたい
start = time.time()
times = []


for page in range(1, max_page + 1):
    # ページネーション用のクエリパラメータを追加
    url = f'{base_url}&page={page}'
    soup = load_page(url)
    before = time.time()
    # 物件情報リストを指定
    mother = soup.find_all(class_='cassetteitem')
        
    # 物件ごとの処理
    for child in mother:

        # 建物情報
        data_home = []
        # カテゴリ
        data_home.append(child.find(class_='ui-pct ui-pct--util1').text)
        # 建物名
        data_home.append(child.find(class_='cassetteitem_content-title').text)
        # 住所
        data_home.append(child.find(class_='cassetteitem_detail-col1').text)
        # 最寄り駅のアクセス
        children = child.find(class_='cassetteitem_detail-col2')
        for id,grandchild in enumerate(children.find_all(class_='cassetteitem_detail-text')):
            data_home.append(grandchild.text)
        # 築年数と階数
        children = child.find(class_='cassetteitem_detail-col3')
        for grandchild in children.find_all('div'):
            data_home.append(grandchild.text)

        # 部屋情報
        rooms = child.find(class_='cassetteitem_other')
        for room in rooms.find_all(class_='js-cassette_link'):
            data_room = []
            
            # 部屋情報が入っている表を探索
            for id_, grandchild in enumerate(room.find_all('td')):
                # 階
                if id_ == 2:
                    data_room.append(grandchild.text.strip())
                # 家賃と管理費
                elif id_ == 3:
                    data_room.append(grandchild.find(class_='cassetteitem_other-emphasis ui-text--bold').text)
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--administration').text)
                # 敷金と礼金
                elif id_ == 4:
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--deposit').text)
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--gratuity').text)
                # 間取りと面積
                elif id_ == 5:
                    data_room.append(grandchild.find(class_='cassetteitem_madori').text)
                    data_room.append(grandchild.find(class_='cassetteitem_menseki').text)
                # url
                elif id_ == 8:
                    get_url = grandchild.find(class_='js-cassette_link_href cassetteitem_other-linktext').get('href')
                    abs_url = urllib.parse.urljoin(url,get_url)
                    data_room.append(abs_url)
            # 物件情報と部屋情報をくっつける
            data_sample = data_home + data_room
            data_samples.append(data_sample)
    
    # 1アクセスごとに1秒休む
    time.sleep(1)
    
    # 進捗確認
    # このページの作業時間を表示
    after = time.time()
    running_time = after - before
    times.append(running_time)
    print(f'{page}ページ目：{running_time}秒')
    # 取得した件数
    print(f'総取得件数：{len(data_samples)}')
    # 作業進捗
    complete_ratio = round(page/max_page*100,3)
    print(f'完了：{complete_ratio}%')
    # 作業の残り時間目安を表示
    running_mean = np.mean(times)
    running_required_time = running_mean * (max_page - page)
    hour = int(running_required_time/3600)
    minute = int((running_required_time%3600)/60)
    second = int(running_required_time%60)
    print(f'残り時間：{hour}時間{minute}分{second}秒\n')

# 処理時間を測りたい
finish = time.time()
running_all = finish - start
print('総経過時間：',running_all)

1ページ目：1.243589162826538秒
総取得件数：116
完了：1.0%
残り時間：0時間2分3秒

2ページ目：1.2404110431671143秒
総取得件数：221
完了：2.0%
残り時間：0時間2分1秒

3ページ目：1.1861603260040283秒
総取得件数：283
完了：3.0%
残り時間：0時間1分58秒

4ページ目：1.2520291805267334秒
総取得件数：352
完了：4.0%
残り時間：0時間1分58秒

5ページ目：1.1960129737854004秒
総取得件数：409
完了：5.0%
残り時間：0時間1分56秒

6ページ目：1.4082942008972168秒
総取得件数：470
完了：6.0%
残り時間：0時間1分57秒

7ページ目：1.2409842014312744秒
総取得件数：541
完了：7.0%
残り時間：0時間1分56秒

8ページ目：1.4296870231628418秒
総取得件数：615
完了：8.0%
残り時間：0時間1分57秒

9ページ目：1.3709588050842285秒
総取得件数：695
完了：9.0%
残り時間：0時間1分56秒

10ページ目：1.2105600833892822秒
総取得件数：762
完了：10.0%
残り時間：0時間1分55秒

11ページ目：1.1743171215057373秒
総取得件数：820
完了：11.0%
残り時間：0時間1分52秒

12ページ目：1.1907849311828613秒
総取得件数：883
完了：12.0%
残り時間：0時間1分51秒

13ページ目：1.1606950759887695秒
総取得件数：937
完了：13.0%
残り時間：0時間1分49秒

14ページ目：1.1641850471496582秒
総取得件数：995
完了：14.0%
残り時間：0時間1分47秒

15ページ目：1.134695291519165秒
総取得件数：1044
完了：15.0%
残り時間：0時間1分45秒

16ページ目：1.1823680400848389秒
総取得件数：1099
完了：16.0%
残り時間：0時間1分43秒

17ページ目：1.6511478424072266秒
総取得件数：1224
完了：17.

In [12]:
# data_samplesをDataFrameに変換
columns = ['property_type', 'building_name', 'address', 'access_1', 'access_2', 'access_3',
    'age', 'building_floors', 'room_floor', 'rent', 'management_fee',
    'deposit', 'gratuity', 'layout', 'area', 'url']
df = pd.DataFrame(data_samples, columns=columns)

# 現在の日付を掲載日として入力
current_date = date.today()
df['posting_date'] = current_date

In [13]:
# 重複行を削除
df_unique = df.drop_duplicates()

# df_unique

In [14]:
# 前処理

# 共益費management_feeと敷金(deposit)と礼金(gratuity)が「-」の場合は0とする
df_unique['management_fee'] = df_unique['management_fee'].replace('-', '0円')
df_unique['deposit'] = df_unique['deposit'].replace('-', '0万円')
df_unique['gratuity'] = df_unique['gratuity'].replace('-', '0万円')

# 金額関連の列を数値に変換する関数
def yen_to_float(yen_str):
    return float(yen_str.replace('万円', '')) * 10000

# rent, management_fee, deposit, gratuity を数値に変換
df_unique['rent'] = df_unique['rent'].apply(yen_to_float)
df_unique['management_fee'] = df_unique['management_fee'].str.replace('円', '').astype(float)
df_unique['deposit'] = df_unique['deposit'].apply(yen_to_float)
df_unique['gratuity'] = df_unique['gratuity'].apply(yen_to_float)

# 面積を数値に変換
df_unique['area'] = df_unique['area'].str.replace('m2', '').astype(float)

# 実質負担額を計算（rent + management_fee + (deposit + gratuity) / 48）
df_unique['actual_cost'] = df_unique['rent'] + df_unique['management_fee'] + (df_unique['deposit'] + df_unique['gratuity']) / 48

In [16]:
df_unique.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2073 entries, 0 to 2072
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   property_type    2073 non-null   object 
 1   building_name    2073 non-null   object 
 2   address          2073 non-null   object 
 3   access_1         2073 non-null   object 
 4   access_2         2073 non-null   object 
 5   access_3         2073 non-null   object 
 6   age              2073 non-null   object 
 7   building_floors  2073 non-null   object 
 8   room_floor       2073 non-null   object 
 9   rent             2073 non-null   float64
 10  management_fee   2073 non-null   float64
 11  deposit          2073 non-null   float64
 12  gratuity         2073 non-null   float64
 13  layout           2073 non-null   object 
 14  area             2073 non-null   float64
 15  url              2073 non-null   object 
 16  posting_date     2073 non-null   object 
 17  actual_cost   

# データの保存

In [17]:
df_unique.to_csv('scraped_data.csv')

# データベース関連の処理

In [61]:
# CSVファイルを読み込む
df = pd.read_csv('scraped_data.csv', index_col=0)

In [70]:
import configparser

config = configparser.ConfigParser()
config.read('config.ini')

db_user = config['database']['user']
db_password = config['database']['password']
db_host = config['database']['host']
db_name = config['database']['name']
db_port = config['database']['port']

In [71]:
# PostgreSQLデータベースへの接続情報
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# dbを読み込み
db_df = pd.read_sql_table('rental_properties', engine)

In [72]:
df.head(2)

,property_type,building_name,address,access_1,access_2,access_3,age,building_floors,room_floor,rent,management_fee,deposit,gratuity,layout,area,url,posting_date,actual_cost
0,賃貸アパート,地下鉄桜通線 桜山駅 2階建 築8年,愛知県名古屋市瑞穂区駒場町４,地下鉄桜通線/桜山駅 歩3分,地下鉄桜通線/瑞穂区役所駅 歩9分,地下鉄鶴舞線/御器所駅 歩18分,築8年,2階建,1階,53500.0,4000.0,0.0,53500.0,1K,20.13,https://suumo.jp/chintai/jnc_000062206827/?bc=...,2024-02-25,58614.583333
1,賃貸アパート,地下鉄桜通線 桜山駅 2階建 築8年,愛知県名古屋市瑞穂区駒場町４,地下鉄桜通線/桜山駅 歩3分,地下鉄桜通線/瑞穂区役所駅 歩9分,地下鉄鶴舞線/御器所駅 歩18分,築8年,2階建,1階,53500.0,4000.0,0.0,53500.0,1K,20.13,https://suumo.jp/chintai/jnc_000088370773/?bc=...,2024-02-25,58614.583333


In [73]:
db_df.head(2)

,id,property_type,building_name,address,access_1,access_2,access_3,age,building_floors,room_floor,...,management_fee,deposit,gratuity,layout,area,url,posting_date,removed_date,actual_cost,predict_cost


In [74]:
# 新規レコードの挿入
## スクレイピングしたデータ（df）にあって、データベース（db_df）にないレコードを特定
new_records = df.merge(db_df, on=['url'], how='left', indicator=True).loc[lambda x : x['_merge']=='left_only']

## 不要なカラムを削除
new_records = new_records.drop(columns=['_merge'])

## 新規レコードに掲載日を設定
#new_records['posting_date'] = current_date

# カラム名を修正するためのrename操作
new_records.rename(columns={
    'property_type_x': 'property_type',
    'building_name_x': 'building_name',
    'address_x': 'address',
    'access_1_x': 'access_1',
    'access_2_x': 'access_2',
    'access_3_x': 'access_3',
    'age_x': 'age',
    'building_floors_x': 'building_floors',
    'room_floor_x': 'room_floor',
    'rent_x': 'rent',
    'management_fee_x': 'management_fee',
    'deposit_x': 'deposit',
    'gratuity_x': 'gratuity',
    'layout_x': 'layout',
    'area_x': 'area',
    'url_x': 'url',
    'posting_date_x': 'posting_date',
    'actual_cost_x': 'actual_cost'
}, inplace=True)

# 例: 不要なカラムを削除し、必要なカラムのみを選択
cleaned_df = new_records[['property_type', 'building_name', 'address', 'access_1', 'access_2', 'access_3', 'age', 'building_floors', 'room_floor', 'rent', 'management_fee', 'deposit', 'gratuity', 'layout', 'area', 'url', 'posting_date', 'actual_cost']].copy()

## 新規レコードをデータベースに挿入
cleaned_df.to_sql('rental_properties', engine, if_exists='append', index=False)

In [75]:
cleaned_df.head(2)

,property_type,building_name,address,access_1,access_2,access_3,age,building_floors,room_floor,rent,management_fee,deposit,gratuity,layout,area,url,posting_date,actual_cost
0,賃貸アパート,地下鉄桜通線 桜山駅 2階建 築8年,愛知県名古屋市瑞穂区駒場町４,地下鉄桜通線/桜山駅 歩3分,地下鉄桜通線/瑞穂区役所駅 歩9分,地下鉄鶴舞線/御器所駅 歩18分,築8年,2階建,1階,53500.0,4000.0,0.0,53500.0,1K,20.13,https://suumo.jp/chintai/jnc_000062206827/?bc=...,2024-02-25,58614.583333
1,賃貸アパート,地下鉄桜通線 桜山駅 2階建 築8年,愛知県名古屋市瑞穂区駒場町４,地下鉄桜通線/桜山駅 歩3分,地下鉄桜通線/瑞穂区役所駅 歩9分,地下鉄鶴舞線/御器所駅 歩18分,築8年,2階建,1階,53500.0,4000.0,0.0,53500.0,1K,20.13,https://suumo.jp/chintai/jnc_000088370773/?bc=...,2024-02-25,58614.583333


In [76]:
# 削除されたレコードの特定と更新
## データベースにあって、スクレイピングしたデータにないレコードを特定
removed_records = db_df.merge(df, on=['url'], how='left', indicator=True).loc[lambda x : x['_merge'] == 'left_only']

# removed_dateがNULL（つまり、未設定）のレコードに対してのみ更新を行う
for index, row in removed_records.iterrows():
    # removed_dateが未設定かどうかを確認するためのクエリ
    check_query = text("SELECT removed_date FROM rental_properties WHERE url = :url AND removed_date IS NULL")
    result = engine.execute(check_query, url=row['url']).fetchone()
    
    # removed_dateが未設定の場合のみ、removed_dateを更新する
    if result:
        update_query = text("UPDATE rental_properties SET removed_date = :current_date WHERE url = :url AND removed_date IS NULL")
        engine.execute(update_query, current_date=current_date, url=row['url'])

# 機械学習モデルによる予測値の入力

In [77]:
# dbを読み込み
df = pd.read_sql_table('rental_properties', engine)
ml_df = pd.read_sql_table('rental_properties', engine)

## 前処理

In [78]:
# 桜山駅からの最短時間を取得する関数
def get_min_time_to_sakurayama(access_str):
    if pd.isnull(access_str):
        return np.nan
    times = [int(s.split('歩')[1].replace('分', '')) for s in access_str.split('/') if '桜山駅' in s]
    if times:
        return min(times)
    else:
        return np.nan

# access_1, access_2, access_3から桜山駅までの最短時間を計算
ml_df['min_time_to_sakurayama'] = ml_df[['access_1', 'access_2', 'access_3']].apply(
    lambda x: min(
        filter(pd.notnull, [get_min_time_to_sakurayama(x['access_1']), get_min_time_to_sakurayama(x['access_2']), get_min_time_to_sakurayama(x['access_3'])]),
        default=np.nan
    ),
    axis=1
)

In [79]:
# '築新'または'新'を'0'年として扱うための処理を追加
ml_df['age'] = ml_df['age'].str.replace('築新', '0').str.replace('新', '0').str.replace('築', '').str.replace('年', '')


# 空の値を0に置き換え
ml_df['age'] = ml_df['age'].replace('', '0')

# 整数型に変換
ml_df['age'] = ml_df['age'].astype(int)

# 建物の階数と部屋の階についても同様に処理
ml_df['building_floors'] = ml_df['building_floors'].str.extract('(\d+)')[0].fillna('0').astype(int)
ml_df['room_floor'] = ml_df['room_floor'].str.extract('(\d+)')[0].fillna('0').astype(int)

In [80]:
# 不要なカラムを削除
drop_cols = ['posting_date', 'removed_date', 'building_name', 'access_1', 'access_2', 'access_3', 'posting_date']
ml_df.drop(drop_cols, axis=1, inplace=True)

In [81]:
ml_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2073 entries, 0 to 2072
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      2073 non-null   int64  
 1   property_type           2073 non-null   object 
 2   address                 2073 non-null   object 
 3   age                     2073 non-null   int64  
 4   building_floors         2073 non-null   int64  
 5   room_floor              2073 non-null   int64  
 6   rent                    2073 non-null   float64
 7   management_fee          2073 non-null   float64
 8   deposit                 2073 non-null   float64
 9   gratuity                2073 non-null   float64
 10  layout                  2073 non-null   object 
 11  area                    2073 non-null   float64
 12  url                     2073 non-null   object 
 13  actual_cost             2073 non-null   float64
 14  predict_cost            0 non-null      

In [82]:
# エンコーディングするカラム
categorical_cols = ['property_type', 'address', 'layout']

# One-hotエンコーディング
ml_df = pd.get_dummies(ml_df, columns=categorical_cols)

## モデルでの予測

In [83]:
ml_df.columns = ml_df.columns.astype(str)

# 特徴量とターゲット変数に分割
X_new = ml_df.drop(['id','url', 'predict_cost'], axis=1)  # ,'predict_cost'# actual_cost以外のすべてのカラムを特徴量として使用
y_new = ml_df['actual_cost']  # actual_costをターゲット変数として使用

In [84]:
# 保存されたモデルをロードする
random_forest_model = load('random_forest_model.joblib')
gradient_boosting_model = load('gradient_boosting_model.joblib')

# ランダムフォレストモデルで予測する
predictions_rf = random_forest_model.predict(X_new)

# グラディエントブースティングモデルで予測する
predictions_gb = gradient_boosting_model.predict(X_new)

# 予測結果をml_dfに追加
ml_df['predict_cost'] = (predictions_rf + predictions_gb) / 2

In [85]:
from sqlalchemy.sql import text

# データベースにデータを格納する（例: 'rental_properties' テーブルを更新）
for index, row in ml_df.iterrows():
    # ここでは、URLを一意の識別子と仮定して、そのURLに対応するレコードを更新します。
    update_query = text("""
    UPDATE rental_properties
    SET actual_cost = :actual_cost, predict_cost = :predict_cost
    WHERE url = :url
    """)
    engine.execute(update_query, {'actual_cost': row['actual_cost'], 'predict_cost': row['predict_cost'], 'url': row['url']})
